 Step 1: Install Required Libraries

What this does: It brings in the tools (libraries) that help us read stories (dialogues) and tell what they mean (summarize).

In [2]:
!pip install transformers datasets sentencepiece


Step 2: Load the DailyDialog Dataset

Now let’s go to the Hugging Face library and grab the DailyDialog book full of short chats.


In [23]:
from datasets import load_dataset

# Load from Hugging Face
dataset = load_dataset("daily_dialog")

# Show one sample
print(dataset["test"][3])


{'dialog': ['Believe it or not , tea is the most popular beverage in the world after water . ', ' Well , people from Asia to Europe all enjoy tea . ', ' Right . And China is the homeland of tea . ', " Yes , Chinese people love drinking tea so much . Some even claim they can't live without tea . ", ' Do you know there are several catagories of Chinese tea ? ', ' Yes , I believe there are green teas , black teas and scented teas . Any Others ? ', ' Well , have you ever heard of Oulong tea and compressed tea ? ', " Oh , yeah . Oulong tea is good for one's health . isn't it ? ", ' You surely know a lot about Chinese tea . ', ' Sure , I like drinking tea at teahouses . ', ' Oh , so do I . ', " Why don't we go for one now ? ", ' Great . We can chat while enjoying a cup there . ', " Let's go ! "], 'act': [1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 3, 4, 3], 'emotion': [0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 4, 4]}


Step 3: Format the Dialogue

The dialogue is like puzzle pieces. We want to make it look like a real conversation with Speaker 1 and Speaker 2.

In [24]:
def format_dialogue(dialogue_lines):
    return "\n".join([f"Speaker {i % 2 + 1}: {line}" for i, line in enumerate(dialogue_lines)])

# Use one example
sample_dialogue = format_dialogue(dataset["test"][3]["dialog"])
print(sample_dialogue)


Speaker 1: Believe it or not , tea is the most popular beverage in the world after water . 
Speaker 2:  Well , people from Asia to Europe all enjoy tea . 
Speaker 1:  Right . And China is the homeland of tea . 
Speaker 2:  Yes , Chinese people love drinking tea so much . Some even claim they can't live without tea . 
Speaker 1:  Do you know there are several catagories of Chinese tea ? 
Speaker 2:  Yes , I believe there are green teas , black teas and scented teas . Any Others ? 
Speaker 1:  Well , have you ever heard of Oulong tea and compressed tea ? 
Speaker 2:  Oh , yeah . Oulong tea is good for one's health . isn't it ? 
Speaker 1:  You surely know a lot about Chinese tea . 
Speaker 2:  Sure , I like drinking tea at teahouses . 
Speaker 1:  Oh , so do I . 
Speaker 2:  Why don't we go for one now ? 
Speaker 1:  Great . We can chat while enjoying a cup there . 
Speaker 2:  Let's go ! 


Step 4: Prompt Templates (Zero, One, Few-Shot)

Just like asking a smart robot:
"Hey, can you tell me what this chat is about in a few words?"

We write different ways to ask it (these are called “prompts”):

What this does: It simply asks, “Please summarize this!”

We also have smarter ways, where we show examples first. That’s called one-shot and few-shot — like giving the robot a few warm-up questions.

In [25]:
def zero_shot_prompt(dialogue):
    return f"Summarize this conversation:\n{dialogue}"

def one_shot_prompt(dialogue):
    example = (
        "Summarize this conversation:\n"
        "Speaker 1: I lost my phone.\n"
        "Speaker 2: Did you try calling it?\n"
        "Speaker 1: Yes, but it's not ringing.\n"
        "Speaker 2: Maybe it's on silent.\n"
        "Summary: A person lost their phone and tried calling it, but it might be on silent.\n\n"
    )
    return example + f"Conversation:\n{dialogue}\nSummary:"

def few_shot_prompt(dialogue):
    examples = (
        "Summarize this conversation:\n"
        "Speaker 1: I'm going to the gym.\n"
        "Speaker 2: That's great! Stay fit!\n"
        "Summary: A person is planning to go to the gym.\n\n"
        "Speaker 1: I feel sick today.\n"
        "Speaker 2: You should rest.\n"
        "Summary: One person feels sick and is advised to rest.\n\n"
    )
    return examples + f"Conversation:\n{dialogue}\nSummary:"


Step 5: Load flan-t5-base Model

We now bring in the robot (the FLAN-T5 model), so it can read and understand.

In [26]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


Step 6: Run Summarization

Now we send the chat to the robot, and it gives us a short summary.

What this does: Converts our question into robot language → asks the robot → then changes the answer back into normal words

In [27]:
import torch

def generate_summary(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)


Step 7: Generate All 3 Types of Summaries

In [29]:
# Format the dialogue
dialogue_text = format_dialogue(dataset["test"][3]["dialog"])

# Zero-shot
zs_prompt = zero_shot_prompt(dialogue_text)
zs_summary = generate_summary(zs_prompt)

# One-shot
os_prompt = one_shot_prompt(dialogue_text)
os_summary = generate_summary(os_prompt)

# Few-shot
fs_prompt = few_shot_prompt(dialogue_text)
fs_summary = generate_summary(fs_prompt)

# Display
print("🧊 Zero-Shot Summary:\n", zs_summary)
print("\n🎯 One-Shot Summary:\n", os_summary)
print("\n🔥 Few-Shot Summary:\n", fs_summary)


🧊 Zero-Shot Summary:
 We will go to teahouses .

🎯 One-Shot Summary:
 People from Asia to Europe all enjoy tea .

🔥 Few-Shot Summary:
 People from Asia to Europe all enjoy tea .


🧊 Zero-Shot: The robot is just told to summarize with no help.

🎯 One-Shot: You show one example first.

🔥 Few-Shot: You show two or three examples so it understands better.